In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

In [2]:
#Code from class resources
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))
session = driver.session(database="neo4j")
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()

In [3]:
#Creating a csv of all stations

rollback_before_flag = True
rollback_after_flag = True

query = """

select station
from stations
order by station

"""

stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations_df.to_csv('stations.csv', index = False)
stations = np.array(stations_df['station'])
stations_df

,station
0,12th Street
1,16th Street Mission
2,19th Street
3,24th Street Mission
4,Antioch
5,Ashby
6,Balboa Park
7,Bay Fair
8,Berryessa
9,Castro Valley


In [4]:
#slightly modified shortest path query from example
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        return int(r['totalCost'])

In [5]:
#Generating all pairs shortest paths csv, probably better to do this all at once with a larger query
stations = np.array(stations_df.station)
start_station = []
end_station = []
weights = []
for i in range(len(stations)):
    for j in range(len(stations)):
        start_station.append(stations[i])
        end_station.append(stations[j])
        weights.append(my_neo4j_shortest_path('depart ' + stations[i], 'arrive ' + stations[j]))
        print(i, j, end = "\r")

In [6]:
#Displaing the csv
shortest_paths_df = pd.DataFrame({"Station 1" : start_station, "Station 2" : end_station, "Time" : weights})
shortest_paths_df

,Station 1,Station 2,Time
0,12th Street,12th Street,0
1,12th Street,16th Street Mission,1140
2,12th Street,19th Street,120
3,12th Street,24th Street Mission,1260
4,12th Street,Antioch,3480
...,...,...,...
2495,West Oakland,Union City,2160
2496,West Oakland,Walnut Creek,1740
2497,West Oakland,Warm Springs,2820
2498,West Oakland,West Dublin,2220


In [7]:
#Saving it for later use
shortest_paths_df.to_csv("shortest_paths.csv", index = False)

In [8]:
#Box code from class
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [11]:
#Getting longitude and latitude of each station
rollback_before_flag = True
rollback_after_flag = True


query = """

select station, latitude, longitude
from stations
order by station

"""

stations_coords = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations_coords

,station,latitude,longitude
0,12th Street,37.803608,-122.272006
1,16th Street Mission,37.764847,-122.420042
2,19th Street,37.807869,-122.268980
3,24th Street Mission,37.752000,-122.418700
4,Antioch,37.996281,-121.783404
5,Ashby,37.853068,-122.269957
6,Balboa Park,37.721667,-122.447500
7,Bay Fair,37.697000,-122.126500
8,Berryessa,37.368361,-121.874655
9,Castro Valley,37.690748,-122.075679


In [9]:
#Adapted query from class but now it has euclidean distance in to station in terms of longitude/latitude minutes
def get_zips_in_vicinity(latitude, longitude, miles):
    rollback_before_flag = True
    rollback_after_flag = True

    latitude = float(latitude)
    longitude = float(longitude)
    point = (latitude, longitude)
    (left, right, top, bottom) = my_calculate_box(point, miles)

    query = "select zip, population, ((latitude - " + str(latitude) + ")^2 + (longitude - " + str(longitude) + ")^2)^.5 as distance "
    query += "from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by (latitude - " + str(latitude) + ")^2 + (longitude - " + str(longitude) + ")^2 asc" 


    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [12]:
#Running query for all stations
s = []
z = []
d = []
p = []

for i in range(len(stations_coords['station'])):
    station = stations_coords['station'][i]
    longitude = stations_coords['longitude'][i]
    latitude = stations_coords['latitude'][i]
    zip_results = get_zips_in_vicinity(latitude, longitude, 2)
    for j in range(len(zip_results['zip'])):
        s.append(station)
        z.append(zip_results['zip'][j])
        d.append(zip_results['distance'][j])
        p.append(zip_results['population'][j])

In [13]:
#Converting results to a DataFrame
proximate_zips = pd.DataFrame({"Station" : s, "Zip Code" : z, "Distance" : d, "Population" : p})
proximate_zips = proximate_zips.sort_values(["Zip Code", "Distance"], ascending = True)
proximate_zips.index = range(len(s))
proximate_zips

,Station,Zip Code,Distance,Population
0,South San Francisco,94005,0.043546,4692
1,Colma,94014,0.019610,49515
2,Daly City,94014,0.026348,49515
3,South San Francisco,94014,0.026859,49515
4,Colma,94015,0.014814,64887
...,...,...,...,...
211,Berryessa,95112,0.025541,61060
212,Berryessa,95116,0.028443,56481
213,Milpitas,95131,0.024016,31170
214,Berryessa,95131,0.029470,31170


In [14]:
#Filtering so zip code is only connected to the closest station
proximate_zips_filtered = proximate_zips.drop_duplicates(subset = ['Zip Code'])
proximate_zips_filtered

,Station,Zip Code,Distance,Population
0,South San Francisco,94005,0.043546,4692
1,Colma,94014,0.019610,49515
4,Colma,94015,0.014814,64887
7,Millbrae,94030,0.016411,22710
9,San Bruno,94066,0.021584,43124
...,...,...,...,...
210,Berryessa,95110,0.041422,20203
211,Berryessa,95112,0.025541,61060
212,Berryessa,95116,0.028443,56481
213,Milpitas,95131,0.024016,31170


In [15]:
#Finding aggregate populations for each station
station_populations = proximate_zips_filtered.groupby(["Station"]).sum(['Population'])
station_populations = station_populations.drop(columns = ["Distance"])
station_populations.to_csv("stations_populations.csv")
station_populations

,Population
Station,
16th Street Mission,78993
19th Street,47558
24th Street Mission,109524
Antioch,66933
Balboa Park,106589
Bay Fair,93041
Berryessa,166470
Castro Valley,44272
Civic Center,177424


In [16]:
#Suplementing last DataFrame with 0 values for stations with no zip codes to which they are closest, converting to dictionary for quick indexing later
pop_dict = {}
for i in range(len(station_populations.index)):
    pop_dict[station_populations.index[i]] = station_populations['Population'][i]
for i in range(len(stations)):
    if not stations[i] in pop_dict.keys():
        pop_dict[stations[i]] = 0 
pop_dict

{'16th Street Mission': 78993,
 '19th Street': 47558,
 '24th Street Mission': 109524,
 'Antioch': 66933,
 'Balboa Park': 106589,
 'Bay Fair': 93041,
 'Berryessa': 166470,
 'Castro Valley': 44272,
 'Civic Center': 177424,
 'Coliseum': 36148,
 'Colma': 114402,
 'Concord': 28428,
 'Daly City': 31488,
 'Downtown Berkeley': 65838,
 'Dublin': 54420,
 'El Cerrito Plaza': 32956,
 'El Cerrito del Norte': 39962,
 'Embarcadero': 15839,
 'Fremont': 73855,
 'Fruitvale': 52299,
 'Glen Park': 72373,
 'Hayward': 66056,
 'Lafayette': 29639,
 'Lake Merritt': 102146,
 'MacArthur': 53100,
 'Millbrae': 22710,
 'Milpitas': 31170,
 'Montgomery Street': 48914,
 'North Berkeley': 36008,
 'North Concord': 58689,
 'OAK': 14619,
 'Orinda': 19341,
 'Pittsburg Center': 96081,
 'Pleasant Hill': 35096,
 'Powell Street': 29689,
 'Richmond': 71468,
 'Rockridge': 30406,
 'SFO': 135,
 'San Bruno': 110721,
 'San Leandro': 82681,
 'South Hayward': 93452,
 'South San Francisco': 4692,
 'Union City': 74601,
 'Walnut Creek': 

In [18]:
#Adapted query from example
def next_step(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        return r['nodes']

In [19]:
#Mapping influence in terms of next station along a shortest path, weighted by population of origin node
stations = np.array(stations_df.station)
start_station = []
start_point = []
end_point = []
next_station = []
weights = []
for i in range(len(stations)):
    for j in range(len(stations)):
        if stations[i] != stations[j]:
            last_station = stations[i]
            ns = next_step('depart ' + stations[i], 'arrive ' + stations[j])
            for n in ns:
                node = " ".join(n.split(" ")[1:])
                if node != last_station:
                    start_station.append(last_station)
                    start_point.append(stations[i])
                    end_point.append(stations[j])
                    next_station.append(node)
                    last_station = node
                    weights.append(pop_dict[stations[i]])

In [20]:
#Converting to DataFrame
package_influence = pd.DataFrame({"Start" : start_point, "End" : end_point, "Origin Station" : start_station, "Destination Station" : next_station, "Weight" : weights})
package_influence

,Start,End,Origin Station,Destination Station,Weight
0,12th Street,16th Street Mission,12th Street,West Oakland,0
1,12th Street,16th Street Mission,West Oakland,Embarcadero,0
2,12th Street,16th Street Mission,Embarcadero,Montgomery Street,0
3,12th Street,16th Street Mission,Montgomery Street,Powell Street,0
4,12th Street,16th Street Mission,Powell Street,Civic Center,0
...,...,...,...,...,...
25967,West Oakland,West Dublin,Fruitvale,Coliseum,26254
25968,West Oakland,West Dublin,Coliseum,San Leandro,26254
25969,West Oakland,West Dublin,San Leandro,Bay Fair,26254
25970,West Oakland,West Dublin,Bay Fair,Castro Valley,26254


In [23]:
#Aggregating by origin and destination stations
pi_df = package_influence.groupby(["Origin Station", "Destination Station"]).sum()
pi_df

Weight
Origin Station      Destination Station          
12th Street         19th Street          37006020
                    Lake Merritt         12080944
                    West Oakland         11325885
16th Street Mission 24th Street Mission  20144835
                    Civic Center         26065080
...                                           ...
West Dublin         Castro Valley         2612160
                    Dublin                2756529
West Oakland        12th Street          18045193
                    Embarcadero          26424384
                    Lake Merritt         15195952

[102 rows x 1 columns]

In [24]:
#Saving results
pi_df.to_csv("package_influence.csv")